# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f21e4f695b0>
├── 'a' --> tensor([[ 1.1069,  0.7683, -1.8001],
│                   [-0.8279, -0.5462,  0.9078]])
└── 'x' --> <FastTreeValue 0x7f21e4f69cd0>
    └── 'c' --> tensor([[ 0.0266, -0.0732, -0.0118,  0.4925],
                        [-0.2249, -0.1677,  1.1172,  0.3157],
                        [ 0.2862,  0.2572, -0.5545,  0.2737]])

In [4]:
t.a

tensor([[ 1.1069,  0.7683, -1.8001],
        [-0.8279, -0.5462,  0.9078]])

In [5]:
%timeit t.a

59.3 ns ± 0.0713 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f21e4f695b0>
├── 'a' --> tensor([[-0.2260, -0.5159, -0.7075],
│                   [-1.0928, -0.0710, -0.7186]])
└── 'x' --> <FastTreeValue 0x7f21e4f69cd0>
    └── 'c' --> tensor([[ 0.0266, -0.0732, -0.0118,  0.4925],
                        [-0.2249, -0.1677,  1.1172,  0.3157],
                        [ 0.2862,  0.2572, -0.5545,  0.2737]])

In [7]:
%timeit t.a = new_value

58.7 ns ± 0.0335 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1069,  0.7683, -1.8001],
               [-0.8279, -0.5462,  0.9078]]),
    x: Batch(
           c: tensor([[ 0.0266, -0.0732, -0.0118,  0.4925],
                      [-0.2249, -0.1677,  1.1172,  0.3157],
                      [ 0.2862,  0.2572, -0.5545,  0.2737]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1069,  0.7683, -1.8001],
        [-0.8279, -0.5462,  0.9078]])

In [11]:
%timeit b.a

57.7 ns ± 0.0207 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.4910,  1.8747,  0.9439],
               [-0.3469,  0.3348,  0.8213]]),
    x: Batch(
           c: tensor([[ 0.0266, -0.0732, -0.0118,  0.4925],
                      [-0.2249, -0.1677,  1.1172,  0.3157],
                      [ 0.2862,  0.2572, -0.5545,  0.2737]]),
       ),
)

In [13]:
%timeit b.a = new_value

484 ns ± 0.195 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

831 ns ± 0.242 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 44.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 658 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 697 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f2137bd5250>
├── 'a' --> tensor([[[ 1.1069,  0.7683, -1.8001],
│                    [-0.8279, -0.5462,  0.9078]],
│           
│                   [[ 1.1069,  0.7683, -1.8001],
│                    [-0.8279, -0.5462,  0.9078]],
│           
│                   [[ 1.1069,  0.7683, -1.8001],
│                    [-0.8279, -0.5462,  0.9078]],
│           
│                   [[ 1.1069,  0.7683, -1.8001],
│                    [-0.8279, -0.5462,  0.9078]],
│           
│                   [[ 1.1069,  0.7683, -1.8001],
│                    [-0.8279, -0.5462,  0.9078]],
│           
│                   [[ 1.1069,  0.7683, -1.8001],
│                    [-0.8279, -0.5462,  0.9078]],
│           
│                   [[ 1.1069,  0.7683, -1.8001],
│                    [-0.8279, -0.5462,  0.9078]],
│           
│                   [[ 1.1069,  0.7683, -1.8001],
│                    [-0.8279, -0.5462,  0.9078]]])
└── 'x' --> <FastTreeValue 0x7f2137bd51f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 71.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f21e4f86190>
├── 'a' --> tensor([[ 1.1069,  0.7683, -1.8001],
│                   [-0.8279, -0.5462,  0.9078],
│                   [ 1.1069,  0.7683, -1.8001],
│                   [-0.8279, -0.5462,  0.9078],
│                   [ 1.1069,  0.7683, -1.8001],
│                   [-0.8279, -0.5462,  0.9078],
│                   [ 1.1069,  0.7683, -1.8001],
│                   [-0.8279, -0.5462,  0.9078],
│                   [ 1.1069,  0.7683, -1.8001],
│                   [-0.8279, -0.5462,  0.9078],
│                   [ 1.1069,  0.7683, -1.8001],
│                   [-0.8279, -0.5462,  0.9078],
│                   [ 1.1069,  0.7683, -1.8001],
│                   [-0.8279, -0.5462,  0.9078],
│                   [ 1.1069,  0.7683, -1.8001],
│                   [-0.8279, -0.5462,  0.9078]])
└── 'x' --> <FastTreeValue 0x7f21e4f86f40>
    └── 'c' --> tensor([[ 0.0266, -0.0732, -0.0118,  0.4925],
                        [-0.2249, -0.1677,  1.1172,  0.3157],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 69 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.7 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0266, -0.0732, -0.0118,  0.4925],
                       [-0.2249, -0.1677,  1.1172,  0.3157],
                       [ 0.2862,  0.2572, -0.5545,  0.2737]],
              
                      [[ 0.0266, -0.0732, -0.0118,  0.4925],
                       [-0.2249, -0.1677,  1.1172,  0.3157],
                       [ 0.2862,  0.2572, -0.5545,  0.2737]],
              
                      [[ 0.0266, -0.0732, -0.0118,  0.4925],
                       [-0.2249, -0.1677,  1.1172,  0.3157],
                       [ 0.2862,  0.2572, -0.5545,  0.2737]],
              
                      [[ 0.0266, -0.0732, -0.0118,  0.4925],
                       [-0.2249, -0.1677,  1.1172,  0.3157],
                       [ 0.2862,  0.2572, -0.5545,  0.2737]],
              
                      [[ 0.0266, -0.0732, -0.0118,  0.4925],
                       [-0.2249, -0.1677,  1.1172,  0.3157],
                       [ 0.2862,  0.2572, -0.5545,  0.2737]],

In [26]:
%timeit Batch.stack(batches)

77.9 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0266, -0.0732, -0.0118,  0.4925],
                      [-0.2249, -0.1677,  1.1172,  0.3157],
                      [ 0.2862,  0.2572, -0.5545,  0.2737],
                      [ 0.0266, -0.0732, -0.0118,  0.4925],
                      [-0.2249, -0.1677,  1.1172,  0.3157],
                      [ 0.2862,  0.2572, -0.5545,  0.2737],
                      [ 0.0266, -0.0732, -0.0118,  0.4925],
                      [-0.2249, -0.1677,  1.1172,  0.3157],
                      [ 0.2862,  0.2572, -0.5545,  0.2737],
                      [ 0.0266, -0.0732, -0.0118,  0.4925],
                      [-0.2249, -0.1677,  1.1172,  0.3157],
                      [ 0.2862,  0.2572, -0.5545,  0.2737],
                      [ 0.0266, -0.0732, -0.0118,  0.4925],
                      [-0.2249, -0.1677,  1.1172,  0.3157],
                      [ 0.2862,  0.2572, -0.5545,  0.2737],
                      [ 0.0266, -0.0732, -0.0118,  0.4925],
                   

In [28]:
%timeit Batch.cat(batches)

160 µs ± 248 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

307 µs ± 4.41 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
